In [1]:
from datasets import load_dataset
dataset = load_dataset("rajpurkar/squad")
    # dataset = load_dataset("quac")

    # train and validation
train = dataset['train']
validation = dataset['validation']

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import getopt
import sys

from kwQnA._exportPairs import exportToJSON
from kwQnA._getentitypair import GetEntity
from kwQnA._graph import GraphEnt
from kwQnA._qna import QuestionAnswer

import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm






class Main:
    """docstring for Main."""

    def __init__(self):
        super(Main, self).__init__()
        self.qna = QuestionAnswer()
        self.getEntity = GetEntity()
        self.export = exportToJSON()
        self.graph = GraphEnt()

    def main(self, argv):
        inputfile = ''
        inputQue = ''
        try:
            opts, args = getopt.getopt(argv, "hi:q:g:s:", ["ifile=", "question=","showGraph=","showEntities="])
            if opts == [] and args == []:
                print("ERROR")
                print("Help:")
                print("python init.py -i <TextFileName> -q <Question> -s <show Ent>")
        except getopt.GetoptError as err:
            sys.exit(2)

        for opt, arg in opts:
            showGraph , showEntities= "f", "f"
            if opt == '-h':
                print ('test.py -i <inputfile> -q <question> -g <y or n> -s <Show Entities>')
                sys.exit()
            elif opt in ("-i", "--ifile"):
                inputfile = arg
            elif opt in ("-q", "--question"):
                inputQue = arg
            elif opt in ("-g", "--showGraph"):
                showGraph = arg
            elif opt in ("-s", "--showEntities"):
                showEntities = arg
            else:
                assert False, "unhandled option"

        return inputfile, inputQue, showGraph, showEntities


def QuestionStartsWith_Accuracy(initialize, dataset, startsWith):
    # to lower
    correct = 0
    total = 0
    for item in tqdm(dataset):
        try:
            context = item['context'].lower()
            question = item['question'].lower()
            question = question.replace("manys", "many")
            # re.sub(r'manys', 'many', question)
            question = re.sub(' +', ' ', question)
            answer = item['answers']['text'][0].lower()
            title = item['title'].lower()
            startsWith = startsWith.lower()
            # check if question starts with startsWith
            if question.startswith(startsWith):
                total += 1
                # if (total < 893):
                #     continue
                question = question.replace("news papers", "newspapers")

                # if (total in (893, 905)):
                #     print("Debug")
                    
                refined_text = initialize.getEntity.preprocess_context(context, title)
                dataEntities, numberOfPairs = initialize.getEntity.get_entity(refined_text)
                if dataEntities:
                    initialize.export.dumpdata(dataEntities[0])

                    outputAnswer = initialize.qna.findanswer(question.lower(), numberOfPairs)
                    if outputAnswer in answer or answer in outputAnswer:
                        correct += 1

                    # else:
                    #     print("Wrong:", total)
                    #     print("Question: ", question)
                    #     print("Answer: ", answer)
                    #     print("Output: ", outputAnswer)
                    #     print("\n\n")

                
                if total % 100 == 0:
                    print(f"Accuracy of {startsWith}: {100*correct/total}")
                    print(f"Correct: {correct}, out of {total}")
                # else:
                #     print("Not Applicable - mfeesh entities")
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Error: ", e)
            # total -= 1
            print("\n\n")
    if total != 0:
        print(f"Accuracy of {startsWith}: {100*correct/total}")
        print(f"Correct: {correct}, out of {total}")
    else:
        print("No Questions found with the given starting word")


if __name__ == "__main__":
    initialize = Main()

    # dataset = load_dataset("rajpurkar/squad")
    # # dataset = load_dataset("quac")

    # # train and validation
    # train = dataset['train']
    # validation = dataset['validation']


    QuestionStartsWith_Accuracy(initialize, train, "who")
    # count = 0
    # titles = set()
    # max_context = 0
    # max_contexts = 0
    # prev_title = ""
    # for item in train:
    #     if prev_title != item['title']:
    #         context = item['context']
    #     elif item['context'] not in context:
    #         context += " " + item['context']
    #     max_context = max(max_context, len(item['context']))
    #     max_contexts = max(max_contexts, len(context))
    #     if (max_contexts == 92637):
    #         print(context)
    #         break
    #     prev_title = item['title']
    #     titles.add(item['title'])
    #     context = item['context'].lower()
    #     question = item['question'].lower()
    #     answer = item['answers']['text'][0].lower()
    #     if (answer in context):
    #         count += 1
    # total = len(train)
    # print(total)
    # print(count)
    # print(len(titles))
    # print(max_context)
    # print(max_contexts)



  9%|▉         | 7949/87599 [00:00<00:07, 10050.55it/s]

Debug
Accuracy of who: 0.0
Correct: 0, out of 900
Debug


 10%|█         | 8946/87599 [00:39<18:10, 72.09it/s]   

Accuracy of who: 0.9
Correct: 9, out of 1000


 12%|█▏        | 10177/87599 [01:16<24:41, 52.25it/s]

...

Accuracy of who: 5.0
Correct: 115, out of 2300


 29%|██▉       | 25777/87599 [08:02<43:54, 23.46it/s]  

Accuracy of who: 5.125
Correct: 123, out of 2400


 31%|███       | 26896/87599 [08:33<19:19, 52.34it/s]  


KeyboardInterrupt: 

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
# dataset = load_dataset("quac")

# train and validation
train = dataset['train']
validation = dataset['validation']

# save them to csv
# train.to_csv('dataset/train.csv')
# validation.to_csv('dataset/validation.csv')


In [ ]:
import json 
import regex as re
import pandas as pd

context_question_answer = []
for item in train:
    context = item['context']
    question = item['question']
    answers = item['answers']
    # context_question_answer.append({'context':context, 'question':question, 'answers':answers})
    
    break

In [ ]:
def QuestionStartsWith_Accuracy(dataset, startsWith):
    # to lower
    for item in dataset:
        context = item['context'].lower()
        question = item['question'].lower()
        answer = item['answer'].lower()
        startsWith = startsWith.lower()
        
        # check if question starts with startsWith
        if question.startswith(startsWith):
            refined_text = initialize.getEntity.preprocess_conttext(input_file, inputQue)
            dataEntities, numberOfPairs = initialize.getEntity.get_entity(refined_text)

        

In [ ]:
import json 
import regex as re
import pandas as pd
# load train and validation from csv
train = pd.read_csv('dataset/train.csv')
validation = pd.read_csv('dataset/validation.csv')

In [ ]:
def FormatAnswer(answer):
    print(answer)
    answer = re.sub(r'array\(\[(\d+)\]\)', r'[\1]', answer)

    answer = answer.replace("'texts'", '"texts"')
    answer = answer.replace("array([", "[")
    answer = answer.replace("\\'", "'")
    # remove unnecessary spaces with regex
    answer = re.sub(r'\s+', ' ', answer)
    answer = answer.replace(", dtype=object)","")

    print(answer)
    # '{"texts": [["Malayalam is the language spoken by the Malayalis."], ["Malayalam is derived from old Tamil and Sanskrit in the 6th century."] , ["Malayalam literature is ancient in origin. The oldest literature works in Malayalam, distinct from the Tamil tradition,"] , ["The oldest literature works in Malayalam, distinct from the Tamil tradition, is dated between the 9th century and 11th century. Malayalam literature includes the 14th century Niranam poets ("] , ["Madhava Panikkar, Sankara Panikkar and Rama Panikkar), whose works mark the dawn of both modern Malayalam language and indigenous Keralite poetry."] , ["All his works are written between 1829 and 1870. Chavara"s contribution to Malayalam literature includes, Chronicles, Poems - athmanuthapam (compunction of the soul),"] , [". In the second half of the 20th century, Jnanpith awardees like G. Sankara Kurup, S. K. Pottekkatt, Thakazhi Sivasankara Pillai and M. T. Vasudevan Nair and non Jnanpith"] ], "answer_starts": [[0], [51], [478], [521], [711], [1283], [1580]]}'
    answer = json.loads(answer)
    return answer['texts'][0]

def FormatQuestion(question):
    question = question[1:-1].replace("?'", "?',")      # adds comma after each question
    question = question.replace("'", "")                # remove single quotes
    question = question.replace('"', "")                # remove double quotes
    question = question.replace("\n", "")               # remove new line

    # parse question to list
    questions = list(question.split(","))
    return questions[0]


In [ ]:
# Make a csv containing the context, formatted question and formatted answer
i = 0
for item in train.iterrows():
    if i == 1:
        question = FormatQuestion(item[1]['questions'])
        answer = FormatAnswer(item[1]['answers'])
        context = item[1]['context']
        print("Question: ", question)
        print("Answer: ", answer)
        print("Context: ", context)
        print("\n")

    i += 1
    
    


In [ ]:

train.head()

print(train['context'][0])
print(train['questions'][0])
question = train['questions'][0][1:-1].replace("?'", "?',")
print(question)
question = question.replace("'", "")        # remove single quotes
question = question.replace('"', "")        # remove double quotes
question = question.replace("\n", "")       # remove new line

# parse question to list
quesitons = list(question.split(","))
print(quesitons)
# transform train['questions'][0] to list
# train['questions'] = train['questions'].apply(lambda x: json.loads(x))
# print(train['questions'][0])

answer = train['answers'][0]
# Change array([51]) to [51] using regex, and remove the ) at the end of the string
answer = re.sub(r'array\(\[(\d+)\]\)', r'[\1]', answer)

answer = answer.replace("\'", "\"")
answer = answer.replace("array([", "[")
# remove unnecessary spaces with regex
answer = re.sub(r'\s+', ' ', answer)
answer = answer.replace(", dtype=object)","")

# print(answer)
answer = json.loads(answer)
print(answer)

# print(answer['texts'][0])


